In [3]:
# 读入模块和文本数据
import pandas as pd
import numpy as np
import openpyxl
import jieba
from jieba import analyse
from jieba import posseg
import paddle
import re

"""
数据名：吉林省医保智慧医保智能客服平台热线工单数据
数据量：91376
采集时间：2022/06/01-2023/03/05
采集指标：['工单属性', '工单编号', '状态', '工单来源', '当前处理人', '当期处理部门', '工单类型', '受理时间', '主叫电话',
       '联系电话', '事项分类', '紧急程度', '处理期限', '完结时间', '处理时限', '诉求来源', '诉求人名称', '涉事标题',
       '主体类型', '涉事主体', '诉求内容', '答复意见', '补充信息', '是否需要书面回复', '是否无效', '是否回访',
       '是否保密', '附件数量']
任务目标1：提取源数据中'涉事标题'和'诉求内容'文本数据，通过清洗，分词，去停用词，提取特征词等步骤初步了解医保事务中的热点话题
任务目标2：知识图谱？贝叶斯网络？
"""

df = pd.read_excel("data/客服人员记录的工单-20220601-20230305.xlsx")

In [ ]:
# 文本清洗、分词、去停用词、提取与医保事务相关的特征词
"""
涉事标题：短文本，内容部分规整，基本形式为“关于xxx的问题”或者直接为“xxx”，存在较多缺失值(29610)
诉求内容：长文本，内容少数规整，多数填写较短，且存在部分无效填写或反映信息较少的情况，如“xxx接听”

目标：就涉事标题和诉求内容分开进行处理，首先去除句中的多余空格和换行符，然后结合jieba中通用词典和自定义词典进行分词，同时做去停用词处理。
将句子的分词结果按列表另存于txt文件中，最后再根据TextRank算法提取特征词

特点：分词时采用了百度paddle（飞桨）深度学习模块训练序、列标注网络模型来实现分词，同时采用隐马尔可夫模型(HMM)识别词典中不存在的新词
"""
length = df.shape[0] # 文本条数

# 1.文本清洗+分词+去停用词+保存（不含词性标注版本）
f1 = open("data/涉事标题.txt",'a+',encoding='utf-8') # 涉事标题
f2 = open("data/诉求内容.txt",'a+',encoding='utf-8') # 诉求内容
f3 = open("data/stopwords.txt","r",encoding='utf-8') # 读取停用词表（原停用词表共1895词）
stopwords = f3.read().split("\n")

for i in range(length):
    """
    涉事标题
    """
    str1 = df['涉事标题'][i] # 读入句子
    str1 = re.sub('\s*|\\n',"",str1) # 剔除句中多余的空格，换行符等等
    jieba.load_userdict('data/words.txt') # 除通用词典外，加载自定义词典
    seg_list1 = jieba.cut(str1, cut_all=False, use_paddle=True, HMM=True) # 分词并返回迭代器（采用paddle精确分词模式，允许识别新词）
    seg_list1 = ("/".join(seg_list1)).split('/') # 将分词结果存入列表
    for j in range(len(seg_list1)): # 去停用词并将结果写入文档
        if seg_list1[j] not in stopwords:
            f1.write(str(seg_list1[j])+" ") 
    f1.write("\n")
    """
    诉求内容
    """
    str2 = df['诉求内容'][i] 
    str2 = re.sub('\s*|\\n',"",str2) 
    jieba.load_userdict('data/words.txt') 
    seg_list2 = jieba.cut(str2, cut_all=False, use_paddle=True, HMM=True) 
    seg_list2 = ("/".join(seg_list2)).split('/')
    for k in range(len(seg_list2)): 
        if seg_list2[k] not in stopwords:
            f2.write(str(seg_list2[k])+" ") 
    f2.write("\n")

f1.close()
f2.close()
f3.close()

# 2.文本清洗+分词+去停用词+保存（含词性标注版本）
from jieba import posseg
f11 = open("data/涉事标题X.txt",'a+',encoding='utf-8') 
f22 = open("data/诉求内容X.txt",'a+',encoding='utf-8') 
f3 = open("data/stopwords.txt","r",encoding='utf-8') 
stopwords = f3.read().split("\n")

for i in range(length):
    """
    涉事标题
    """
    str1 = df['涉事标题'][i] 
    str1 = re.sub('\s*|\\n',"",str1) 
    jieba.load_userdict('data/words.txt') 
    seg_list1 = posseg.cut(str1, use_paddle=True, HMM=True) # 分词并返回迭代器（迭代器包含分词及对应词性）
    seg_list1 = list(seg_list1) # 将分词结果转为列表
    for j in range(len(seg_list1)): # 去停用词并将结果写入文档
        if list(seg_list1[j])[0] not in stopwords:
            f11.write(str(list(seg_list1[j]))+" ") 
    f11.write("\n")
    """
    诉求内容
    """
    str2 = df['诉求内容'][i] 
    str2 = re.sub('\s*|\\n',"",str2) 
    jieba.load_userdict('data/words.txt') 
    seg_list2 = posseg.cut(str2, use_paddle=True, HMM=True) 
    seg_list2 = list(seg_list2) # 将分词结果转为列表
    for j in range(len(seg_list2)): # 去停用词并将结果写入文档
        if list(seg_list2[j])[0] not in stopwords:
            f22.write(str(list(seg_list2[j]))+" ") 
    f22.write("\n")

f11.close()
f22.close()
f3.close()

In [4]:
# 提取关键词并制作词云图
"""
jieba中analyse集成了分词，词性标注和关键词提取三个功能，使用十分方便。其中关键词提取分TF-IDF和TextRank两种提取思

【1】TF-IDF
其中TF为词频，指某个词在当前文档（已分词）的出现频率；IDF为逆文档频率，指包含该词的文档数占总文档数比例的倒数。
可见TF和IDF越大，越有可能为关键词，在analyse模块中，已内置通用文档语料库，IDF值已经给定

【2】TextRank
TextRank算法是一种文本排序算法，由谷歌的网页重要性排序算法PageRank算法改进而来。分词完成后，TextRank算法基于相邻位置滑动窗口和频数计算，
求得两个词之间的相似性。最后把词化作图的节点，相似性化作图的边，利用PageRank算法迭代计算每个词的rank,最后对rank排序以返回关键词

在实际测试中，TF-IDF提取法的效果往往更好，一方面是利用了内置语料库中的IDF信息，另一方面TextRank算法对于待提取文本质量往往有更高要求
此外，在进行分词与去停用词后，提取效果也会更好

云图的制作借助了线上词云网站：易词云 https://www.yciyun.com/
"""
# 涉事标题
text1 = open("data/涉事标题.txt","r",encoding='utf-8').read()
keywords11 = analyse.extract_tags(text1, topK=30, withWeight=True, allowPOS=('n', 'v')) # TF-IDF（词频*逆文档频率）提取法
keywords12 = analyse.textrank(text1, topK=30, withWeight=True, allowPOS=('n', 'v')) # TextRank提取法
print('TF-IDF提取法'+"\t"+'TextRank提取法')
for i in range(30):
    print(keywords11[i],'\t',keywords12[i])

# 诉求内容
text2 = open("data/诉求内容.txt","r",encoding='utf-8').read()
keywords21 = analyse.extract_tags(text2, topK=30, withWeight=True, allowPOS=('n', 'v')) 
keywords22 = analyse.textrank(text2, topK=30, withWeight=True, allowPOS=('n', 'v'))
print('TF-IDF提取法'+"\t"+'TextRank提取法')
for i in range(30):
    print(keywords21[i],'\t',keywords22[i])

Building prefix dict from the default dictionary ...
[2023-09-05 15:52:10,701] [   DEBUG] __init__.py:113 - Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\XGQ\AppData\Local\Temp\jieba.cache
[2023-09-05 15:52:10,703] [   DEBUG] __init__.py:132 - Loading model from cache C:\Users\XGQ\AppData\Local\Temp\jieba.cache
Loading model cost 0.915 seconds.
[2023-09-05 15:52:11,617] [   DEBUG] __init__.py:164 - Loading model cost 0.915 seconds.
Prefix dict has been built successfully.
[2023-09-05 15:52:11,621] [   DEBUG] __init__.py:166 - Prefix dict has been built successfully.


TF-IDF提取法	TextRank提取法
('异地', 0.8821972230099264) 	 ('异地', 1.0)
('就医', 0.6560433194917332) 	 ('就医', 0.7056341064716433)
('备案', 0.46587173931408965) 	 ('备案', 0.5942878064930227)
('参保', 0.35703466836509046) 	 ('查询', 0.3636616178097616)
('报销', 0.2762280062140543) 	 ('报销', 0.35695631483500556)
('查询', 0.21839318751230605) 	 ('参保', 0.3433842718086795)
('特病', 0.19857033705011312) 	 ('办理', 0.24139158362621796)
('医保卡', 0.17375554014855812) 	 ('特病', 0.2007685574267731)
('门诊', 0.11717505818696936) 	 ('医保卡', 0.18455689056504054)
('待遇', 0.1155907252896146) 	 ('账户', 0.16592486668724693)
('办理', 0.11256458201395744) 	 ('单位', 0.15291740261060435)
('基数', 0.10410686021071112) 	 ('待遇', 0.15166533685864494)
('特药', 0.09177489637169035) 	 ('门诊', 0.15130419629551847)
('在职', 0.09155591671309479) 	 ('业务', 0.13976281807593946)
('缴费', 0.09152681447178922) 	 ('退休', 0.1358607345205681)
('定点', 0.0877335803142852) 	 ('定点', 0.12723211874520102)
('账户', 0.08685785710979996) 	 ('在职', 0.12616825980978655)
('退休', 0.08175139

d:\miniconda\envs\pytorch\lib\site-packages\jieba\posseg\__init__.py:16: DeprecationWarning: invalid escape sequence '\.'
  re_skip_detail = re.compile("([\.0-9]+|[a-zA-Z0-9]+)")
d:\miniconda\envs\pytorch\lib\site-packages\jieba\posseg\__init__.py:17: DeprecationWarning: invalid escape sequence '\.'
  re_han_internal = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._]+)")
d:\miniconda\envs\pytorch\lib\site-packages\jieba\posseg\__init__.py:18: DeprecationWarning: invalid escape sequence '\s'
  re_skip_internal = re.compile("(\r\n|\s)")
d:\miniconda\envs\pytorch\lib\site-packages\jieba\posseg\__init__.py:21: DeprecationWarning: invalid escape sequence '\.'
  re_num = re.compile("[\.0-9]+")


KeyboardInterrupt: 

In [119]:
# 关键词聚类
"""
关键词表已由上述提取过程给出

关键词聚类算法：
1.相似性度量：假设每条文本工单代表一个完整的语义段（即事件表述完整），那么在同一条工单中两个词的同时出现某种意义上可以认为具有关联性，
注：考虑到词向量矩阵稀疏的问题，故采取每10条工单词向量进行相加合并，即稀疏矩阵压缩

2.聚类算法：余弦相似度+Kmeans聚类算法，聚类数目拟定为4或5
"""
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import AgglomerativeClustering


keywords_list1 = ['异地','就医','备案','参保','报销','查询','特病','医保卡','门诊','待遇','办理','基数','特药',
'在职','缴费','定点','账户','退休','催办','补缴','共济','征缴','额度','接续','来电','余额','业务','新增','报错','退保']

keywords_list2 = ['参保','异地','就医','备案','报销','查询','来电','办理','告知','办结','特病','医保卡','待遇','门诊',
'经办人','接听','单位','缴费','基数','解答','定点','特药','账户','退休','在职','补缴','催办','征缴','报错','业务']

length = df.shape[0]
length_dense = length//10

# 词向量矩阵构建与压缩
# 涉事标题
words1 = [] # 词库
word_fea1 = None # 词向量特征
wordvec1 = None # 词向量矩阵转置（仅包含关键词）
wordvec1_dense = np.full((30,length_dense),0) # 压缩矩阵
text1 = open("data/涉事标题.txt","r",encoding='utf-8') # 涉事标题分词文本
for i in range(length):
    temp = []
    lines = text1.readline().split(" ")[:-1]
    for j in range(len(lines)):
        if lines[j] in keywords_list1:
            temp.append(lines[j])
    words1.append(" ".join(temp))
model1 = CountVectorizer()
bag1 = model1.fit_transform(words1) # bag是一个稀疏的矩阵。因为词袋模型就是一种稀疏的表示。
word_fea1 = dict(zip(model1.vocabulary_.values(), model1.vocabulary_.keys())) # 输出单词与编号的映射关系
wordvec1 = bag1.toarray().T # 调用稀疏矩阵的toarray方法，将稀疏矩阵转换为ndarray对象。
for i in range(30):
    for j in range(length_dense):
        wordvec1_dense[i][j] = np.sum(wordvec1[i,j*10:j*10+10])

Agg = AgglomerativeClustering(n_clusters=5, metric='cosine',linkage='complete').fit(wordvec1_dense)
label1 = Agg.labels_

print("涉事标题关键词聚类")
for i in range(5):
    print("第"+str(i+1)+'类关键词包含：',end=" ")
    for j in range(30):
        if label1[j]==i:
            print(word_fea1[j],end=" ")
    print("\n")


# 诉求内容
words2 = [] 
word_fea2 = None
wordvec2 = None 
text2 = open("data/诉求内容.txt","r",encoding='utf-8') 
wordvec2_dense = np.full((30,length_dense),0) 
for i in range(length):
    temp = []
    lines = text2.readline().split(" ")[:-1]
    for j in range(len(lines)):
        if lines[j] in keywords_list2:
            temp.append(lines[j])
    words2.append(" ".join(temp))
model2 = CountVectorizer()
bag2 = model2.fit_transform(words2)
word_fea2 = dict(zip(model2.vocabulary_.values(), model2.vocabulary_.keys())) # 输出单词与编号的映射关系
wordvec2 = bag2.toarray().T
for i in range(30):
    for j in range(length_dense):
        wordvec2_dense[i][j] = np.sum(wordvec2[i,j*10:j*10+10])

Agg = AgglomerativeClustering(n_clusters=5, metric='cosine',linkage='complete').fit(wordvec2_dense)
label2 = Agg.labels_

print("诉求内容关键词聚类")
for i in range(5):
    print("第"+str(i+1)+'类关键词包含：',end=" ")
    for j in range(30):
        if label2[j]==i:
            print(word_fea2[j],end=" ")
    print("\n")

涉事标题关键词聚类
第1类关键词包含： 在职 基数 退休 

第2类关键词包含： 办理 备案 定点 就医 异地 征缴 报销 报错 特病 特药 

第3类关键词包含： 业务 余额 催办 待遇 接续 新增 查询 缴费 补缴 账户 门诊 额度 

第4类关键词包含： 共济 

第5类关键词包含： 医保卡 参保 来电 退保 

诉求内容关键词聚类
第1类关键词包含： 在职 基数 征缴 缴费 补缴 退休 

第2类关键词包含： 业务 办理 医保卡 单位 参保 备案 定点 就医 异地 待遇 报销 报错 查询 特病 特药 经办人 账户 门诊 

第3类关键词包含： 办结 告知 来电 解答 

第4类关键词包含： 接听 

第5类关键词包含： 催办 

